In [ ]:
import pandas as pd

In [ ]:
from broker_limit import broker_limit
config_path="config.env"
broker_obj=broker_limit(config_path)
# broker_obj.ssh_host
conn=broker_obj.make_db_connection()

In [ ]:
date_start='2025-07-01'
date_end='2025-07-31'
query = "select * from invoices i where approved_date between %s and %s"
cur = conn.cursor()
cur.execute(query, (date_start, date_end))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
invoice_df=pd.DataFrame(results, columns=col_name)